In [5]:
import io
import os
import re
import sys
import csv
import gzip
import time
import json
import math
import pickle
import secrets
import multiprocessing
import subprocess
import resource
import pandas as pd
# import sqlite3
# from PIL import Image
import matplotlib.pyplot as plt
# from plotnine import *
import numpy as np
import scipy as sp
import statsmodels.api as sm
import random
import itertools
import collections
import pyreadr
import pywfa
from IPython.display import display_html

import multiprocessing as mp
from functools import partial

sys.path.append('/well/band/users/rbx225/software/lcwgsus/')
import lcwgsus
from lcwgsus.variables import *
from warnings import simplefilter
from hla_phase import *
from hla_align_functions import *
from hla_align import *

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

pd.options.mode.chained_assignment = None

In [26]:
def per_read(reads, ref, refseq, q):
    for i, (seq, bq) in enumerate(zip(reads['sequence'], reads['base_quality'])):
        result = ref(seq)
        refseq_aligned = refseq[result.pattern_start:result.pattern_end]
        seq_aligned = seq[result.text_start:result.text_end]
        likelihood_per_read_per_allele1 = calculate_score_per_alignment(seq_aligned, refseq_aligned, bq)
#         scores_mat[i, j] = likelihood_per_read_per_allele
    for i, (seq, bq) in enumerate(zip(reads['rev_seq'], reads['rev_bq'])):
        result = ref(seq)
        refseq_aligned = refseq[result.pattern_start:result.pattern_end]
        seq_aligned = seq[result.text_start:result.text_end]
        likelihood_per_read_per_allele2 = calculate_score_per_alignment(seq_aligned, refseq_aligned, bq)
#         rev_scores_mat[i, j] = likelihood_per_read_per_allele
    return likelihood_per_read_per_allele1, likelihood_per_read_per_allele2

def multi_calculate_loglikelihood_per_read(reads, db, temperature=1):
    reads['rev_seq'] = reads['sequence'].apply(reverse_complement)
    reads['rev_bq'] = reads['base_quality'].apply(lambda bq: bq[::-1])

    scores_mat = np.zeros((reads.shape[0], db.shape[1]))
    rev_scores_mat = np.zeros((reads.shape[0], db.shape[1]))
    
    manager = multiprocessing.Manager()
    q = manager.Queue()
    processes = []
    
    for j, a in enumerate(db.columns[:10]):
        refseq = (''.join(db[a].tolist())).replace('.', '')
        ref = pywfa.WavefrontAligner(refseq)
        for i, (seq, bq) in enumerate(zip(reads['sequence'], reads['base_quality'])):
            tmp = multiprocessing.Process(target=per_read,
                                              args=(reads, ref, refseq, q))
            tmp.start()
            processes.append(tmp)

        for process in processes:
            process.join()
        res_lst = []
        while not q.empty():
            res_lst.append(q.get())
        else:
            for res in res_lst:
                scores_mat[i, j] = res[0]
                rev_scores_mat[i, j] = res[1]

    reads = reads.drop(columns = ['rev_seq', 'rev_bq'])
    scores_mat = np.maximum(scores_mat, rev_scores_mat)
    likelihood_mat = np.exp(scores_mat/temperature)/np.sum(np.exp(scores_mat/temperature), axis = 1, keepdims = True)
    loglikelihood_mat = np.log(likelihood_mat)
    return loglikelihood_mat

In [40]:
def per_allele(j, a, db, reads, q):
    refseq = (''.join(db[a].tolist())).replace('.', '')
    ref = pywfa.WavefrontAligner(refseq)
    scores_mat_ary1 = []
    scores_mat_ary2 = []
    for i, (seq, bq) in enumerate(zip(reads['sequence'], reads['base_quality'])):
        result = ref(seq)
        refseq_aligned = refseq[result.pattern_start:result.pattern_end]
        seq_aligned = seq[result.text_start:result.text_end]
        likelihood_per_read_per_allele1 = calculate_score_per_alignment(seq_aligned, refseq_aligned, bq)
        scores_mat_ary1.append(likelihood_per_read_per_allele1)
    for i, (seq, bq) in enumerate(zip(reads['rev_seq'], reads['rev_bq'])):
        result = ref(seq)
        refseq_aligned = refseq[result.pattern_start:result.pattern_end]
        seq_aligned = seq[result.text_start:result.text_end]
        likelihood_per_read_per_allele2 = calculate_score_per_alignment(seq_aligned, refseq_aligned, bq)
        scores_mat_ary2.append(likelihood_per_read_per_allele2)
    
    q.put([j, np.array(scores_mat_ary1), np.array(scores_mat_ary2)])
    
def multi_calculate_loglikelihood_per_allele(reads, db, temperature=1):
    reads['rev_seq'] = reads['sequence'].apply(reverse_complement)
    reads['rev_bq'] = reads['base_quality'].apply(lambda bq: bq[::-1])

    scores_mat = np.zeros((reads.shape[0], db.shape[1]))
    rev_scores_mat = np.zeros((reads.shape[0], db.shape[1]))
    
    manager = multiprocessing.Manager()
    q = manager.Queue()
    processes = []
    
    for j, a in enumerate(db.columns[:10]):
        tmp = multiprocessing.Process(target=per_allele,
                                          args=(j, a, db, reads, q))
        tmp.start()
        processes.append(tmp)

    for process in processes:
        process.join()
    res_lst = []
    while not q.empty():
        res_lst.append(q.get())
    else:
        for res in res_lst:
            j = res[0]
            scores_mat[:, j] = res[1]
            rev_scores_mat[:, j] = res[2]

    reads = reads.drop(columns = ['rev_seq', 'rev_bq'])
    scores_mat = np.maximum(scores_mat, rev_scores_mat)
    likelihood_mat = np.exp(scores_mat/temperature)/np.sum(np.exp(scores_mat/temperature), axis = 1, keepdims = True)
    loglikelihood_mat = np.log(likelihood_mat)
    return loglikelihood_mat   

In [43]:
# gene = 'DRB1'
# hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')
# bam = "/well/band/users/rbx225/GAMCC/data/bams/IDT0482.bam"
# hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')
# db = pd.read_csv(f'/well/band/users/rbx225/recyclable_files/hla_reference_files/v3570_aligners/{gene}.ssv', sep = ' ')

# reads_apart_max=1000
# temperature=100
# n_mismatches=5
# assumed_bq=0.001

# reads1 = get_chr6_reads(gene, bam, hla_gene_information, reads_apart_max)
# reads2 = get_hla_reads(gene, bam, reads_apart_max)

# if reads1.empty:
#     reads1 = reads2.iloc[:2, :] if not reads2.empty else pd.DataFrame()
# elif reads2.empty:
#     reads2 = reads1.iloc[:2, :]
# else:
#     pass

# rl = reads1['sequence'].str.len().mode().values[0]

start = time.time()
# likemat1 = multi_calculate_loglikelihood_per_allele(reads1, db, temperature = 1)
likemat1 = calculate_loglikelihood(reads1, db, temperature = 1)
end = time.time()
print(end - start)

# likemat2 = calculate_loglikelihood1(reads2, db)
# min_valid_prob = np.log(math.comb(rl, n_mismatches)) + n_mismatches*np.log(assumed_bq) + (rl - n_mismatches)*np.log(1 - assumed_bq)

# valid_indices1 = np.any(likemat1 >= min_valid_prob, axis=1)
# valid_indices2 = np.any(likemat2 >= min_valid_prob, axis=1)
# likemat1, reads1 = likemat1[valid_indices1], reads1[valid_indices1]
# likemat2, reads2 = likemat2[valid_indices2], reads2[valid_indices2]

# likemat_all = np.vstack((likemat1, likemat2))

# id1, id2 = reads1.iloc[:, 0].to_numpy(), reads2.iloc[:, 0].to_numpy()

# readind = (reads1.iloc[:, 1].astype(int) // 64) % 4
# readind2 = (reads2.iloc[:, 1].astype(int) // 64) % 4
# mate_indicator = np.concatenate((readind, readind2))

# ids_all = np.concatenate((id1, id2))
# unique_ids = np.unique(ids_all)
# likemat_mate = np.zeros((len(unique_ids), likemat_all.shape[1]))

# for i, uid in enumerate(unique_ids):
#     t1 = likemat_all[ids_all == uid, :]
#     t2 = mate_indicator[ids_all == uid]
#     if len(t2) > 0:
#         likemat_mate[i, :] = np.sum(t1[t2 > 0], axis=0)

# valid_mask = likemat_mate.max(axis=1) >= min_valid_prob
# likemat_mate = likemat_mate[valid_mask]
# likemat_norm = 0.5 * np.exp(likemat_mate - likemat_mate.max(axis=1, keepdims=True)) + 1e-100

# likemat_paired = likemat_norm.T @ likemat_norm
# likemat_paired = pd.DataFrame(likemat_paired, index=db.columns, columns=db.columns)

# likemat_mate = pd.DataFrame(likemat_mate, index = unique_ids[valid_mask], columns=db.columns)

226.36023259162903


In [51]:
gene = 'DRB1'
hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')
bam = "/well/band/users/rbx225/GAMCC/data/bams/IDT0482.bam"

# db = process_db_genfile(gene, ipd_gen_file_dir, hla_gene_information)
# db.to_csv(f'/well/band/users/rbx225/recyclable_files/hla_reference_files/v3570_aligners/{gene}.ssv', sep = ' ', index = False, header = True)

db = pd.read_csv(f'/well/band/users/rbx225/recyclable_files/hla_reference_files/v3570_aligners/{gene}.ssv', sep = ' ')

r1, r2, mate, pair = hla_aligner(gene, bam, db, hla_gene_information)

r1.to_csv(outdir + '/reads1.csv', header = False, index = False)
r2.to_csv(outdir + '/reads2.csv', header = False, index = False)

pd.set_option('display.float_format', '{:.6e}'.format)
mate.to_csv(outdir + '/mate_likelihood_matrix.ssv', index=True, header=True, sep = ' ')
pair.to_csv(outdir + '/pair_likelihood_matrix.ssv', index=True, header=True, sep = ' ')